In [5]:
import numpy as np
import torch
from torch.nn import functional as F
import transformers
import pdb
from datasets import load_dataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from transformers import GPT2LMHeadModel
from transformers import GPT2TokenizerFast

In [6]:
def get_dataset():
    train_dataset = load_dataset("rotten_tomatoes", split="train")
    eval_dataset = load_dataset("rotten_tomatoes", split="validation")

In [7]:
def load_init_model_and_optimizer():
    pass